In [1]:
from scripts.LabTestAnalysis.machine_learning.LabNormalityPredictionPipeline \
        import NON_PANEL_TESTS_WITH_GT_500_ORDERS
import os
import pandas as pd

import stats_utils
import matplotlib.pyplot as plt

stats_utils.py:111: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'agg' by the following code:
  File "/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/Users/songxu/healthrex/CDSS/env/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/songxu/healthrex/CDSS/env/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/songxu/healthrex/CDSS/env/lib/python2.7/site-packages/ipykernel/kernelapp.py"

In [2]:
result_label="all_labs"
targeted_PPV=0.95
scale_by='enc'
use_cached_fig_data=False
inverse01=True

all_labs = NON_PANEL_TESTS_WITH_GT_500_ORDERS

data_source = 'Stanford'
lab_type = 'panel'
curr_version = '10000-episodes-lastnormal'

In [3]:
stats_folderpath = os.path.join(stats_utils.main_folder, 'lab_statistics/')
ml_folderpath = os.path.join(stats_utils.main_folder, 'machine_learning')

import LocalEnv
statsByDataSet_foldername = 'data-%s-%s-%s' % (data_source, lab_type, curr_version)
statsByDataSet_folderpath = os.path.join(stats_folderpath, statsByDataSet_foldername)

dataset_foldername = 'data-%s-%s-%s'%(data_source, lab_type, curr_version)
labStats_folderpath = os.path.join(LocalEnv.PATH_TO_CDSS, 'scripts/LabTestAnalysis/lab_statistics')
statsByLab_folderpath = os.path.join(labStats_folderpath, dataset_foldername)
ml_folderpath = statsByLab_folderpath.replace("lab_statistics", "machine_learning")

In [4]:
vols_tmp = [
4246,
2244,
2120,
1471,
1230,
1197,
752,
534,
455,
400,
371,
361
]

In [7]:
def draw__Diagnostic_Metrics(result_label='all_labs',
                            targeted_PPV=0.95, scale_by=None, use_cached_fig_data=False,
                             inverse01=False):
    '''
    Drawing Figure 3 in the main text.

    :return:
    '''
    labs = all_labs
    print 'draw__Diagnostic_Metrics for', labs, ' with label %s...'%result_label
    
    labs_stats_filepath = os.path.join(statsByLab_folderpath, 'summary-stats-bestalg-fixTrainPPV.csv')

    df = pd.read_csv(labs_stats_filepath, keep_default_na=False)

    # df = df[df['fixTrainPPV'] == targeted_PPV]

    df = df[df['lab'].isin(labs)]

    cached_foldername = 'Fig3_Diagnostic_Metrics/'
    cached_folderpath = os.path.join(os.path.join(statsByLab_folderpath, cached_foldername))

    cached_tablename = 'Diagnostic_Metrics_%ss_PPV_%.2f__%s.csv'%(lab_type, targeted_PPV, result_label)
    cached_tablepath = os.path.join(cached_folderpath, cached_tablename)

    cached_figurename = 'Diagnostic_Metrics_%ss_PPV_%.2f_ind__%s.png'%(lab_type, targeted_PPV, result_label)
    cached_figurepath = os.path.join(cached_folderpath, cached_figurename)

    if not os.path.exists(cached_folderpath):
        os.mkdir(cached_folderpath)

    if os.path.exists(cached_tablepath) and use_cached_fig_data:
        # lab2stats = pickle.load(open(cached_result_path, 'r'))
        df_toplots = pd.read_csv(cached_tablepath, keep_default_na=False)
        # print df_toplots

    else:
        if data_source == 'Stanford':
            # if 'total_vol' not in df.columns.values.tolist():
            # Stanford data, scaled by vol
            df['total_vol'] = df['total_vol_20140701_20170701']

        elif data_source == 'UCSF':
            import stats_database
            if lab_type == 'panel':
                ucsf_lab_cnt = dict(stats_database.UCSF_PANELS_AND_COUNTS)

            elif lab_type == 'component':
                ucsf_lab_cnt = stats_database.UCSF_COMPONENT_TO_COUNT #dict(stats_database.UCSF_COMPONENTSS_AND_COUNTS)
            df['total_vol'] = df['lab'].apply(lambda x: ucsf_lab_cnt[x])
        elif data_source == 'UMich' and lab_type=='component' and result_label=='important_components':
            df = df[df['lab'].isin(stats_utils.umich_lab_cnt)]


            df['total_vol'] = df['lab'].apply(lambda x: stats_utils.umich_lab_cnt[x])
        else:
            df['total_vol'] = 1



        # TODO: use fractions in the original file!
        df['all_instance'] = df['TP'] + df['FP'] + df['TN'] + df['FN']

        for cnt_type in ['TP', 'FP', 'TN', 'FN']:
            df[cnt_type] = df[cnt_type]/df['all_instance']


        df['all_positive'] = df['TP'] + df['FP']

        # print df[['all_positive', 'total_vol']]
        df['predicted_normal_vol'] = df['all_positive'] * df['total_vol']

        if result_label == 'important_components':

            if data_source == 'UMich' or data_source == 'UCSF':
                df['lab'] = df['lab'].apply(lambda x: map_lab(x, data_source, lab_type, map_type='from_src'))

            df = df[df['lab'].isin(stats_utils.get_important_labs('component'))]
            df['lab'] = pd.Categorical(
                df['lab'],
                categories=stats_utils.get_important_labs('component'),
                ordered=True
            )
            df = df.sort_values('lab')
        elif data_source == 'UMich':
            pass
        else:
            df = df.sort_values('total_vol', ascending=False)

        df['all_negative'] = df['TN'] + df['FN']

        df['TN'] = -df['TN']
        df['all_negative'] = -df['all_negative']

        df_toshow = df.copy().drop_duplicates()

        lab_descriptions = stats_utils.get_lab_descriptions(data_source=data_source,#'Stanford',
                                                                 lab_type=lab_type)
        df_toshow['Lab Test'] = df_toshow['lab'].apply(lambda x:lab_descriptions.get(x,x))
        df_toshow['TN'] = -df_toshow['TN']

        df_toshow['Prev'] = df_toshow['TN'] + df_toshow['FP']
        df_toshow = df_toshow.rename(columns={
            'num_test_episodes':'Count',
                                            'TP':'TP',
                                              'FP':'FP',
                                              'TN':'TN',
                                              'FN':'FN',
                                              'sens':'sens',
                                              'spec':'spec',
                                              'LR_p':'LR+', 'LR_n':'LR-'})

        df_toshow['AUC'] = df_toshow['AUC'].apply(lambda x: '%.2f'%(x))

        print df_toshow['NPV']
        df_toshow['NPV'] = df_toshow['NPV'].apply(lambda x: stats_utils.convert_floatstr2percentage(x))

        numeric_cols = ['Prev', 'PPV', 'TP', 'FP', 'TN', 'FN', 'sens', 'spec']
        for numeric_col in numeric_cols:
            if df_toshow[numeric_col].dtype != type(0.1):
                df_toshow[numeric_col] = df_toshow[numeric_col].apply(lambda x: stats_utils.convert_floatstr2percentage(x))
            else:
                df_toshow[numeric_col] = df_toshow[numeric_col].apply(lambda x: stats_utils.convert_floatnum2percentage(x))

        df_toshow['LR+'] = df_toshow['LR+'].apply(lambda x: stats_utils.convert_floatstr2num(x))
        df_toshow['LR-'] = df_toshow['LR-'].apply(lambda x: stats_utils.convert_floatstr2num(x))

        df_toshow['Vol'] = (df_toshow['total_vol'] / float(stats_utils.NUM_DISTINCT_ENCS / 1000.)).apply(
            lambda x: int(round(x)))
        
        def fix_number(x):
            fix_dict = {4251:4246,
                       2246:2244,
                       2122:2120,
                       1473:1471,
                       1231:1230,
                       1198:1197,
                       753:752,
                       534:534,
                       456:455,
                       401:400,
                       372:371,
                       361:361,}
            return fix_dict.get(x,x)
        df_toshow['Vol'] = df_toshow['Vol'].apply(lambda x: fix_number(x))

        if data_source == 'Stanford' and lab_type=='panel':

            df_toshow.loc[df_toshow['Lab Test']=='Sodium', 'chargemaster'] = 219
            df_toshow.loc[df_toshow['Lab Test'] == 'Specific Gravity', 'medicare'] = 3.28
            df_toshow.loc[df_toshow['Lab Test'] == 'Sepsis Protocol Lactate', 'medicare'] = 11.87
            df_toshow.loc[df_toshow['Lab Test'] == 'LDH Total', 'medicare'] = 6.71
            df_toshow.loc[df_toshow['Lab Test'] == 'Lactate', 'medicare'] = 11.87
            df_toshow.loc[df_toshow['Lab Test'] == 'Urxinalysis', 'medicare'] = 2.67
            df_toshow.loc[df_toshow['Lab Test'] == 'Calcium Ionized', 'medicare'] = 13.73
            df_toshow.loc[df_toshow['Lab Test'] == 'Heparin', 'medicare'] = 16.16

            # print df_toshow
            df_toshow = df_toshow.rename(columns={'medicare': 'Medicare', 'chargemaster': 'Chargemaster'})

            df_toshow['Medicare'] = df_toshow['Medicare'].apply(lambda x: '$%s' % x if x != '' else '-')
            df_toshow['Chargemaster'] = df_toshow['Chargemaster'].apply(lambda x: '$%s' % str(x) if x != '' else '-')

            cols_to_show = ['Lab Test', 'Vol', 'AUC'] \
                           + numeric_cols
                           #+ ['Medicare', 'Chargemaster'] #+ ['LR+', 'LR-'] \
            print df_toshow['Lab Test'].values.tolist()
            labs_to_show = ['Magnesium', 'Prothrombin Time', 'Phosphorus', 'Partial Thromboplastin Time',
                            'Lactate', 'Calcium Ionized', 'Potassium', 'Troponin I', 'LDH Total',
                            'Heparin', 'Urinalysis', # too few positives
                            'Blood Culture (Aerobic & Anaerobic)',
                            'Blood Culture (2 Aerobic)', 'Sodium', 'Lidocaine', 'Hematocrit', 'Urine Culture',
                            'Urinalysis With Microscopic', 'Uric Acid', 'Hemoglobin A1c'
                            ]
        else:
            cols_to_show = ['Lab Test', 'AUC'] + numeric_cols #+ ['LR+', 'LR-']

        if inverse01:
            df_toshow = df_toshow.rename(columns={'PPV': 'NPV',
                                                  'NPV': 'PPV',
                                                  'TP': 'TN',
                                                  'FP': 'FN',
                                                  'TN': 'TP',
                                                  'FN': 'FP',
                                                  'sens':'Spec',
                                                  'spec':'Sens',
                                                  'best_alg':'Best Alg',
                                                  'AUC':'C-stat',
                                                  'fixTrainPPV':'Target NPV'})

            rename_alg = {'xgb': 'xgboost', 'nn': 'neural-nets'}
            df_toshow['Best Alg'] = df_toshow['Best Alg'].apply(lambda x: rename_alg.get(x, x))
            if data_source == 'Stanford':
                cols_to_show = ['lab', 'Lab Test', 'Vol', 'C-stat', 'Prev', 'NPV', 'PPV', 'Sens', 'Spec', 'TN', 'FN', 'TP', 'FP'] #['Lab Test', 'Vol', 'C-stat', 'Prev', 'NPV', 'PPV', 'Sens', 'Spec', 'TN', 'FN', 'TP', 'FP']

            else:
                cols_to_show = ['Lab Test', 'Best Alg', 'C-stat', 'Prev', 'Target NPV', 'NPV', 'PPV', 'Sens',
                                'Spec', 'TN', 'FN', 'TP', 'FP']

        if data_source == 'Stanford':
            if lab_type == 'panel':
                labs_to_show = ['LABMGN', 'LABPT', 'LABPHOS', 'LABPTT', 'LABLACWB', 'LABCAI', 'LABK',
                                'LABTNI', 'LABLDH', 'LABBLC', 'LABBLC2', 'LABNA'#, 'LABHCTX', 'LABURNC'
                               ]
                #['LABMGN', 'LABPHOS', 'LABBLC', 'LABBLC2', 'LABUA', 'LABUAPRN', 'LABLACWB', 'LABCAI'] # suggested by Jason Hom
                df_finalshow = df_toshow[(df_toshow['lab'].isin(labs_to_show)) & (df_toshow['Target NPV']==0.95)]\
                [cols_to_show]
            else:
                labs_to_show = ['WBC', 'HGB', 'NA', 'K', 'CR']
                df_finalshow = df_toshow[df_toshow['lab'].isin(labs_to_show)][cols_to_show]
                
        df_finalshow.to_csv(
                    cached_tablepath.replace('.csv', '_finalshow.csv'), index=False)

        df_toshow[cols_to_show].iloc[:20].to_csv(cached_tablepath.replace('.csv','_toshow.csv'), index=False) #.sort_values('total_vol', ascending=False)

        if data_source == 'Stanford' and lab_type=='panel':
            cols_to_show += ['Medicare', 'Chargemaster']

        df_toshow[cols_to_show].to_csv(cached_tablepath.replace('.csv', '_full.csv'), index=False)

        df['all_positive_vol'] = df['all_positive'] * df['total_vol']
        df['true_positive_vol'] = df['TP'] * df['total_vol']
        df['all_negative_vol'] = df['all_negative'] * df['total_vol']
        df['true_negative_vol'] = df['TN'] * df['total_vol']

        df_toplots = df.iloc[:15]

        df_toplots[['lab',
                    'PPV', 'NPV', 'sens', 'spec', 'LR_p', 'LR_n',
                    'total_vol',
                   'all_positive_vol', 'true_positive_vol', 'all_negative_vol', 'true_negative_vol']]\
                    .to_csv(cached_tablepath, index=False, float_format='%.3f') # .sort_values('total_vol', ascending=False)\

    if not scale_by:
        scale = 1.
    elif scale_by=='pat':
        scale = float(stats_utils.NUM_DISTINCT_PATS/1000.)
    elif scale_by == 'enc':
        scale = float(stats_utils.NUM_DISTINCT_ENCS/1000.)
    elif scale_by == 'enc_ucsf':
        scale = float(stats_utils.NUM_DISTINCT_ENCS_UCSF/1000.)

    if data_source == 'Stanford' or data_source == 'UCSF' or data_source == 'UMich': #True: #lab_type == 'panel':
        df_toplots = df_toplots.iloc[::-1]
    else:
        df_toplots = df_toplots.iloc[::-1]

    for ind, df_toplot in enumerate([df_toplots.tail(38), df_toplots.head(38)]):

        if result_label == 'important_components':
            fig, ax = plt.subplots(figsize=(6, 9))
        else:
            fig, ax = plt.subplots(figsize=(10, 8))

        ax.barh(df_toplot['lab'], df_toplot['all_positive_vol'] / scale, color='orangered', alpha=1,
                label='False Positive')
        ax.barh(df_toplot['lab'], df_toplot['true_positive_vol'] / scale, color='forestgreen', alpha=1,
                label='True Positive')
        ax.barh(df_toplot['lab'], df_toplot['all_negative_vol'] / scale, color='gold', alpha=1,
                label='False Negative')
        ax.barh(df_toplot['lab'], df_toplot['true_negative_vol'] / scale, color='royalblue', alpha=1,
                label='True Negative')

        plt.yticks([])

        if result_label == 'important_components':
            plt.xlim([-8500, 6500])
            plt.xticks([-6000, -3000, 0, 3000, 6000])
            ax.set_xticklabels([6000, 3000, 0, 3000, 6000])

            ax.spines['top'].set_visible(False)
            ax.spines['right'].set_visible(False)
            ax.spines['bottom'].set_visible(True)
            ax.spines['left'].set_visible(False)
            pass
        else:
            for i, v in enumerate(df_toplot['all_positive_vol'] / scale):
                cur_lab = df_toplot['lab'].values[i]
                cur_description = lab_descriptions.get(cur_lab, cur_lab).replace(' - ', '/')

                if data_source == 'UMich':
                    ax.text(v + 0.05, i - 0.2, cur_description, color='k', fontsize=14)
                else:
                    if '\n' in cur_description:
                        ax.text(v + 50, i - 0.3, cur_description, color='k', fontsize=14)
                    else:
                        ax.text(v + 50, i - 0.1, cur_description, color='k', fontsize=14)

            if data_source == 'Stanford' and lab_type == 'panel':
                plt.xlim([-2300, 2700])
            elif data_source == 'Stanford' and lab_type == 'component':
                plt.xlim([-9000, 9000])
            elif data_source == 'UCSF' and lab_type == 'panel':
                plt.xlim([-3200, 3200])
            elif data_source == 'UCSF' and lab_type == 'component':
                plt.xlim([-6000, 6000])
            elif data_source == 'UMich' and lab_type == 'component':
                # plt.xlim([-1.5, 1.5])
                plt.xlim([-8000, 8000])

            if data_source == 'UCSF' or data_source == 'Stanford' or data_source == 'UMich':
                plt.xlabel('Number of orders per 1000 patient encounters', fontsize=18) #, targeting at %.0f'%(targeted_PPV*100)+'% PPV'
            else:
                plt.xlabel('Fraction of orders, targeting at %.0f' % (targeted_PPV * 100) + '% PPV',
                fontsize=18)

        plt.tick_params('x', labelsize=24)


        plt.tight_layout()

        plt.savefig(cached_figurepath.replace('ind', 'ind_%i'%ind))

In [8]:
# statsByDataSet_foldername = 'data-%s-%s-%s' % (self.data_source, self.lab_type, self.curr_version)
# statsByDataSet_folderpath = os.path.join(stats_folderpath, statsByDataSet_foldername)
# if not os.path.exists(statsByDataSet_folderpath):
#     os.mkdir(statsByDataSet_folderpath)
statsByDataSet_folderpath = '.'
draw__Diagnostic_Metrics(result_label=result_label,
                                            targeted_PPV=0.95, scale_by='enc', use_cached_fig_data=False, inverse01=inverse01)

draw__Diagnostic_Metrics for ['LABA1C', 'LABAFBC', 'LABAFBD', 'LABALB', 'LABANER', 'LABB12', 'LABBLC', 'LABBLC2', 'LABBLCSTK', 'LABBLCTIP', 'LABBUN', 'LABBXTG', 'LABCA', 'LABCAI', 'LABCDTPCR', 'LABCK', 'LABCMVQT', 'LABCORT', 'LABCRP', 'LABCSFC', 'LABCSFGL', 'LABCSFTP', 'LABDIGL', 'LABESRP', 'LABFCUL', 'LABFE', 'LABFER', 'LABFIB', 'LABFLDC', 'LABFOL', 'LABFT4', 'LABGRAM', 'LABHAP', 'LABHBSAG', 'LABHCTX', 'LABHEPAR', 'LABHIVWBL', 'LABK', 'LABLAC', 'LABLACWB', 'LABLDH', 'LABLIDOL', 'LABLIPS', 'LABMB', 'LABMGN', 'LABNA', 'LABNH3', 'LABNTBNP', 'LABOSM', 'LABPALB', 'LABPCCG4O', 'LABPCCR', 'LABPCTNI', 'LABPHOS', 'LABPLTS', 'LABPROCT', 'LABPT', 'LABPTEG', 'LABPTT', 'LABRESP', 'LABRESPG', 'LABRETIC', 'LABSPLAC', 'LABSTLCX', 'LABSTOBGD', 'LABTNI', 'LABTRFS', 'LABTRIG', 'LABTSH', 'LABUOSM', 'LABUA', 'LABUAPRN', 'LABUPREG', 'LABURIC', 'LABURNC', 'LABUSPG']  with label all_labs...
176      0.2908204711616572
177      0.3630097645031591
178      0.4320266889074229
179      0.6339285714285714
224    